In [1]:
import json

from carmon import sample_status_pipe
from carmon import common as cmc

Считаем токен из файла

In [2]:
with open("test_data/token.txt", "r") as token_file:
    token = token_file.read()

Объявляем токен для дальнейших настроек

In [3]:
stage_1 = cmc.state_token(token)

In [4]:
stage_1['success']

True

И проверяем факт авторизации

In [5]:
stage_1['payload']['depart_name']

'НИИ Дезинфектологии Роспотребнадзора'

Запрашиваем список возможных статусов, сравниваем его с тем, что записано в конфигураторе. 

In [6]:
stage_2 = sample_status_pipe.request_sample_status_types()

In [7]:
stage_2['success']

True

In [8]:
stage_2['payload']

{'Новый': 1,
 'Отправлен': 2,
 'Доставлен': 3,
 'Брак': 4,
 'В работе': 5,
 'Предварительный результат': 6,
 'Секвенирован': 7,
 'Готов': 8,
 'Принят на секвенирование': 9,
 'Брак сиквенса': 10,
 'Генотипирование по ПЦР': 11}

Тут забавный момент, но если:
* полученный список совпадает с записанным локально, вернется True и дублирование списка;
* полученный список не совпадает с записанным локально, вернется False и список, который следует записать в локальный конфиг.

Считаем ранее подготовленный __df__ с тестовыми образцами. Он выглядит так, как ему следует выглядеть после этапа 1. 

In [9]:
read_stage = cmc.read_df('test_data/test_sample_status_pipe/from_stage_1.csv')

In [10]:
read_stage['success']

True

In [11]:
read_stage['payload']

,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
barcode,,,,,,,,,,,,,,,,,,
barcode01_MN908947.3,2203281,test1,Еврейская автономная область,niid,,,,,NAME AND REGION DUPLICATES,,,,,,,,,
barcode02_MN908947.3,2203282,test2,Еврейская автономная область,niid,6441,НИИ дезинфектологии,niid161SM159359,test2,OK,,,,,,,,,
barcode04_MN908947.3,2203284,test666,Еврейская автономная область,niid,6442,НИИ дезинфектологии,niid161SM159361,test666,OK,,,,,,,,,


Считаем FASTA

In [12]:
stage_3 = sample_status_pipe.state_sample_status_local(read_stage['payload'],
                                                       "test_data/test_sample_status_pipe/all_consensus.fasta")

In [13]:
stage_3['success']

True

Получаем на выходе 2 объекта — таблицу TABLE и словарь с последовательностями

In [14]:
df, fastas = stage_3['payload']

In [15]:
df.head(4)

,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
barcode,,,,,,,,,,,,,,,,,,
barcode01_MN908947.3,2203281,test1,Еврейская автономная область,niid,,,,,NAME AND REGION DUPLICATES,False,Требуется подтверждение,,,,,,,
barcode02_MN908947.3,2203282,test2,Еврейская автономная область,niid,6441,НИИ дезинфектологии,niid161SM159359,test2,OK,False,Брак сиквенса,,,,,,,
barcode04_MN908947.3,2203284,test666,Еврейская автономная область,niid,6442,НИИ дезинфектологии,niid161SM159361,test666,OK,True,Готов,,,,,,,


Запрашиваем с портала _id_ тех образцов, чьи статусы соответствуют возможным на портале статусам. 

In [16]:
stage_4 = sample_status_pipe.request_samples_info(df)

In [17]:
stage_4['success']

True

In [18]:
stage_4['payload']

,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
barcode,,,,,,,,,,,,,,,,,,
barcode01_MN908947.3,2203281,test1,Еврейская автономная область,niid,,,,,NAME AND REGION DUPLICATES,False,Требуется подтверждение,,,,,,,
barcode02_MN908947.3,2203282,test2,Еврейская автономная область,niid,6441,НИИ дезинфектологии,niid161SM159359,test2,OK,False,Брак сиквенса,246951,,,,,,
barcode04_MN908947.3,2203284,test666,Еврейская автономная область,niid,6442,НИИ дезинфектологии,niid161SM159361,test666,OK,True,Готов,246953,,,,,,


Считываем логин и пароль для Basic авторизации

In [19]:
with open("test_data/credentials.json", "r") as ff:
    cre = json.load(ff)

Направляем образцы на загрузку

In [20]:
stage_5 = sample_status_pipe.upload_sequences(stage_4['payload'], fastas, cre)

In [21]:
stage_5['success']

True

In [22]:
stage_5['payload']

,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
barcode,,,,,,,,,,,,,,,,,,
barcode01_MN908947.3,2203281,test1,Еврейская автономная область,niid,,,,,NAME AND REGION DUPLICATES,False,Требуется подтверждение,,,,,,,
barcode02_MN908947.3,2203282,test2,Еврейская автономная область,niid,6441,НИИ дезинфектологии,niid161SM159359,test2,OK,False,Брак сиквенса,246951,,,,,,
barcode04_MN908947.3,2203284,test666,Еврейская автономная область,niid,6442,НИИ дезинфектологии,niid161SM159361,test666,OK,True,Готов,246953,Uploaded,,,,,


Просматриваем статус для загруженных

In [24]:
stage_5['payload'].loc['barcode04_MN908947.3', 'sample_status_remote']

'Uploaded'

Проставляем статус "Брак сиквенса"

In [25]:
stage_5a = sample_status_pipe.state_sample_status_remote(stage_5['payload'])

In [26]:
stage_5a['success']

True

In [27]:
stage_5a['payload']

,litech_barcode,litech_sample_name,litech_region,region_short_name,registry_id,depart_name,sample_number,sample_name_value,registry_guess_status,valid_seq,sample_status_local,sample_vga_id,sample_status_remote,pango,nextclade,sequence_conclusion_local,sequence_vga_id,sequence_conclusion_remote
barcode,,,,,,,,,,,,,,,,,,
barcode01_MN908947.3,2203281,test1,Еврейская автономная область,niid,,,,,NAME AND REGION DUPLICATES,False,Требуется подтверждение,,,,,,,
barcode02_MN908947.3,2203282,test2,Еврейская автономная область,niid,6441,НИИ дезинфектологии,niid161SM159359,test2,OK,False,Брак сиквенса,246951,Проставлено,,,,,
barcode04_MN908947.3,2203284,test666,Еврейская автономная область,niid,6442,НИИ дезинфектологии,niid161SM159361,test666,OK,True,Готов,246953,Uploaded,,,,,


In [31]:
stage_5a['payload'].loc['barcode02_MN908947.3', 'sample_status_remote']

'Проставлено'

Сохраняем для дальнейшего использования

In [35]:
cmc.save_concatenated_table(stage_5a['payload'], "test_data/test_sample_status_pipe/result.csv")

{'success': True,
 'payload': 'Успешно сохранено в `test_data/test_sample_status_pipe/result.csv`'}